In [ ]:
import os
import pandas as pd
import numpy as np
import pandas_gbq
import unidecode

In [ ]:
from google.cloud import bigquery

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='##'

In [ ]:
import firebase_admin
#Firbease (pip install firebase_admin)
from firebase_admin import credentials
from firebase_admin import firestore

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import requests
import yaml

## CUFFLINKS

In [ ]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # requires version >= 1.9.0

In [ ]:
import cufflinks as cf

In [ ]:
init_notebook_mode(connected=True)

In [ ]:
cf.go_offline()

In [ ]:
filename='##'

In [ ]:
# Use a service account
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

In [ ]:
col_query = db.collection('Accounting').stream()

In [ ]:
accounts = []
for acc in col_query:
    accounts.append(acc.to_dict())

In [ ]:
accounting = pd.DataFrame(accounts)

In [ ]:
#accounting.info()

In [ ]:
accounting['month_day']=accounting['execution_date'].apply(lambda x: x[5:10])

In [ ]:
def clean_execution(x):
    if(len(x['execution_date'])<15):
        doc_id = x['id']
        exec_date=x['execution_date']+'T14:14:58.593Z'
        print(exec_date)
        print(len(x['execution_date']))
        a=db.collection(u'Accounting').document(doc_id).update({'execution_date':exec_date})
        
        return exec_date
    else:
        return x['execution_date']
        

In [ ]:
accounting['execution_date']=accounting.apply(lambda x: clean_execution(x) ,axis=1)

In [ ]:
accounting['origin'].value_counts()

In [ ]:
accounting[accounting['origin']=='shopify']['vat'].value_counts()

In [ ]:
def clean_vat_amount(x):
    doc_id = x['id']
    amount=x['amount']
    origin = x['origin']
    if origin =='script_dani':
        if x['vat']>0:
            vat=21
        else:
            vat=0
        #a=db.collection(u'Accounting').document(doc_id).update({'vat':vat})
    else:
        vat=x['vat']
        if not(vat >=0):
            vat=0
    vat_amount=amount*(1-1/(1+vat/100))

    #a=db.collection(u'Accounting').document(doc_id).update({'vat_amount':vat_amount})

    return pd.Series({'vat':vat,'vat_amount':vat_amount})

In [ ]:
nan = accounting[accounting['amount']==304.88]['vat'][23]

In [ ]:
accounting[accounting['amount']==304.88]['vat'][23] 

In [ ]:
42*(1-1/1.21)

In [ ]:
accounting[['vat','vat_amount']] = accounting.apply(lambda x: clean_vat_amount(x),axis=1)

In [ ]:
accounting['creation_date'] = pd.to_datetime(accounting['creation_date'],utc=True)
accounting['execution_date'] = pd.to_datetime(accounting['execution_date'],utc=True)

In [ ]:
accounting.info()

In [ ]:
accounting.head()

In [ ]:
def getimages(x):
    if(len(x)==0):
        return ''
    elif('download_url' in x[0].keys()):
        return x[0]['download_url']
    else:
        return ''

In [ ]:
accounting['receipt']=accounting.images.apply(lambda x: getimages(x))

In [ ]:
sns.heatmap(accounting.isna())

In [ ]:
#accounting[accounting['amount'].isnull()][['id','code']]

In [ ]:
accounting.columns

In [ ]:
df = accounting[['concept', 'place', 'creation_date', 'vat', 'description', 'amount',
       'id', 'phase', 'creator_user', 'execution_date',
       'user_in_charge', 'origin', 'account_id', 'project', 'code', 'type',
       'intervention', 'target_id', 'vat_amount', 'receipt']]#,'month_day']]

## upload to BIGQUERY

In [ ]:
client = bigquery.Client()

In [ ]:
dataset_id = "{}.contoan".format(client.project)

In [ ]:
dataset = bigquery.Dataset(dataset_id)

In [ ]:
table_id = "{}.accounting".format(dataset.dataset_id)

pandas_gbq.to_gbq(df, table_id, project_id=client.project, if_exists='replace')

## get data

In [ ]:
sql = """
    SELECT *
    FROM `oan-nikarit.contoan.accounting`
    LIMIT 100
"""

# Run a Standard SQL query using the environment's default project
df = pd.read_gbq(sql, dialect='standard')

### SUBIR VENTAS

In [ ]:
col_query = db.collection('Nikarit_Sales').stream()

In [ ]:
ventas = []
for acc in col_query:
    ventas.append(acc.to_dict())

In [ ]:
df_ventas = pd.DataFrame(ventas)

In [ ]:
df_ventas.info()

In [ ]:
len(df_ventas[
            (df_ventas['creation_date'].str[0:4]=='2020') &
            (df_ventas['origin']!='Contoan')
         ])

In [ ]:
df_ventas[
            (df_ventas['creation_date'].str[0:4]=='2020')&
            (df_ventas['origin']!='Contoan')
         ]['total_price'].apply(lambda x: float(x)).sum()

In [ ]:
cambios = ['subtotal_price','balsamo_unit_price','packContigo_unit_price','crema_unit_price','packHidra_unit_price',
          'manteca_unit_price','buyer_accepts_marketing']

for cambio in cambios:
    df_ventas[cambio]=pd.to_numeric(df_ventas[cambio])

In [ ]:
df_ventas['creation_date']=pd.to_datetime(df_ventas['creation_date'],utc=True)
df_ventas['close_date']=pd.to_datetime(df_ventas['close_date'],utc=True)

In [ ]:
df_ventas.info()

### METER MUJERES Y HOMBRES

In [ ]:
#client_gender

In [ ]:
url = 'https://raw.githubusercontent.com/marcboquet/spanish-names/master/mujeres.csv'
df_mujeres = pd.read_csv(url, error_bad_lines=False)
l_mujeres = df_mujeres['nombre'].to_list()

In [ ]:
l_mujeres = [str(nom).lower() for nom in l_mujeres]

In [ ]:
url = 'https://raw.githubusercontent.com/marcboquet/spanish-names/master/hombres.csv'
df_hombres = pd.read_csv(url, error_bad_lines=False)
l_hombres = df_hombres['nombre'].to_list()

In [ ]:
l_hombres = [nom.lower() for nom in l_hombres]

In [ ]:
df_ventas['client_name'].value_counts()

In [ ]:
df_ventas['client_gender'] = df_ventas.apply(lambda x: 
                            'F' if unidecode.unidecode(
                                    str(x['client_name']).replace(" ", "").lower()) in l_mujeres
                            else 'M' if unidecode.unidecode(
                                str(x['client_name']).replace(" ", "").lower())  in l_hombres
                            else 'F' if unidecode.unidecode(
                                str(x['client_name']).split(" ")[0].lower())  in l_mujeres
                            else 'M' if unidecode.unidecode(
                                str(x['client_name']).split(" ")[0].lower())  in l_hombres

                            else 'N/A' , axis =1)

In [ ]:
df_ventas['client_gender'].value_counts()

In [ ]:
table_id = "contoan.sales"

pandas_gbq.to_gbq(df_ventas, table_id, project_id=client.project, if_exists='replace')